In [3]:
from google.colab import drive
drive.mount('./drive')

Drive already mounted at ./drive; to attempt to forcibly remount, call drive.mount("./drive", force_remount=True).


In [4]:
import collections
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from pprint import pprint

import nltk
from nltk import stem
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

import os,sys,re
import datetime
dt_now = datetime.datetime.now()
today = dt_now.strftime('%Y%m%d')
nltk.download("stopwords")
nltk.download("punkt")

!pip install polars
import polars as pl

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##  ライブラリのインポート


In [5]:
!pip install transformers
# !pip install pytorch-lightning


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# 各種ライブラリのインポート
import os
import pandas as pd
import itertools
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import DebertaTokenizer, DebertaForMaskedLM
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

## 各種パスの設定

In [7]:
# 各種パスの設定
######################################################################
base_path = '/content/drive/MyDrive/kaggle/probspace' # ベースとなるパスを指定してください。#######
######################################################################
os.makedirs(os.path.join(base_path,'model'), exist_ok=True)  # 学習済みモデルの保存するディレクトリを作成
os.makedirs(os.path.join(base_path,'output'), exist_ok=True)  # 提出用ファイルを出力するディレクトリを作成
train_data_path = os.path.join(base_path,'data/train_data.csv') # 訓練データのパスを指定
test_data_path = os.path.join(base_path,'data/test_data.csv') # テストデータのパスを指定
submit_data_path = os.path.join(base_path,'data/submission.csv') # 提出用サンプルfileのパスを指定
model_file_path = os.path.join(base_path,'model/best_model.pt') # 学習済みモデルのパスを指定
output_file_path = os.path.join(base_path,'output/001_submission.csv') # 提出用ファイルのパスを指定


## 各種データの読み込み

In [8]:
df = pl.read_csv(train_data_path)
df_test = pl.read_csv(test_data_path)
#テストにターゲットのyカラムを追加。（全て0）
df_test = df_test.with_columns(
    pl.DataFrame(np.array([0]*len(df_test)),columns="y")
)
submit_df = pl.read_csv(submit_data_path)

In [9]:
import pandas as pd
import torch
import transformers
import polars as pl

# データを読み込む
df = pd.read_csv(train_data_path)
df = df[:100]

# 入力データとラベルに分割する
inputs = list(df['abstract'])
labels = list(df['y'])

# モデルとトークナイザーを読み込む
tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
model = transformers.RobertaForSequenceClassification.from_pretrained('roberta-base')

# データをトークン化し、モデルに入力する
def tokenize(inputs):
    return tokenizer(inputs, padding=True, truncation=True)

inputs = tokenize(inputs)
inputs = {k: torch.tensor(v) for k, v in inputs.items()}
labels = torch.tensor(labels)

# モデルを訓練する
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(5):
    optimizer.zero_grad()
    outputs = model(**inputs)
    loss = loss_fn(outputs.logits, labels)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} Loss: {loss.item()}")

# モデルの評価
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)

# 結果をPolars DataFrameとして保存する
results = pl.DataFrame({'text': inputs['input_ids'].tolist(), 'label': predictions.tolist()})
results.to_csv('results.csv')


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Epoch 1 Loss: 0.7250445485115051
Epoch 2 Loss: 0.574236273765564
Epoch 3 Loss: 0.37153732776641846
Epoch 4 Loss: 0.12144646048545837
Epoch 5 Loss: 0.033161286264657974


AttributeError: ignored

In [11]:
results

text,label
list[i64],i64
"[0, 170, ... 1]",0
"[0, 38257, ... 1]",0
"[0, 14563, ... 1]",0
"[0, 1121, ... 1]",0
"[0, 170, ... 1]",0
"[0, 1121, ... 1]",0
"[0, 38386, ... 1]",0
"[0, 170, ... 1]",0
"[0, 21001, ... 1]",0


In [39]:
import polars as pl
import torch
from transformers import DebertaTokenizer, DebertaForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split

# データの読み込み
df = df[:100]

# トークナイザーの作成
tokenizer = DebertaTokenizer.from_pretrained('roberta-base')
# config =  DebertaConfig() 
# 入力データとラベルの作成
inputs = tokenizer(list(df['abstract']), padding=True, truncation=True, return_tensors='pt',max_length=402)
labels = torch.tensor(list(df['y']))

# データの分割
train_inputs, test_inputs, train_labels, test_labels = train_test_split(df['abstract'], df['y'], test_size=0.2, random_state=42)

# 入力データとラベルの作成
train_inputs = tokenizer(list(train_inputs), padding=True, truncation=True, return_tensors='pt')
test_inputs = tokenizer(list(train_inputs), padding=True, truncation=True, return_tensors='pt')
train_labels = torch.tensor(list(train_labels))
test_labels = torch.tensor(list(test_labels))

# モデルの作成

model = DebertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# モデルのトレーニング
optimizer = AdamW(model.parameters(), lr=1e-5)
model.train()

num_epochs = 1
for epoch in range(num_epochs):
    optimizer.zero_grad()
    loss, _ = model(train_inputs['input_ids'], attention_mask=train_inputs['attention_mask'], labels=train_labels)
    loss.backward()
    optimizer.step()

# モデルのテスト
model.eval()
with torch.no_grad():
    logits = model(test_inputs['input_ids'], attention_mask=test_inputs['attention_mask'])[0]
    predictions = torch.argmax(logits, dim=-1)
    accuracy = (predictions == test_labels).float().mean().item()

# 結果の表示
print('Accuracy:', accuracy)

ImportError: ignored

## 事前学習

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
#　データセットを作成するクラスを定義します。
class PaperDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.data = dataframe
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        title = self.data[index,1]
        abstract = self.data[index,3] #abstract
        text = title + ' ' + abstract
        label = self.data[index,-1] #label y
        inputs = self.tokenizer(text)
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.float)
        }

In [ ]:
# datasetで定義されたバッチ形式の__getitem__を処理して、オリジナルのtorch.Tensorにする関数
def collate_fn(batch):
    input_ids = pad_sequence([torch.tensor(item["input_ids"]) for item in batch], batch_first=True)
    attention_mask = pad_sequence([torch.tensor(item["attention_mask"]) for item in batch], batch_first=True)
    labels = torch.tensor([item['targets'] for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, 'targets': labels}

In [ ]:
#　BERTのモデルのクラスを定義します。
class BERTModel(nn.Module):
    def __init__(self):
        super(BERTModel, self).__init__()
        self.bert = DebertaTokenizer.from_pretrained('microsoft/deberta-xlarge')
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(768, 1)
        self.sig = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask,token_type_ids=None)
        pooled_output = outputs[1]
        dropout_output = self.dropout(pooled_output)
        output = self.fc(dropout_output)
        return self.sig(output)

## データローダーの作成

In [ ]:
#トレーニングとバリデーションのためのデータローダーを作成します。
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

tokenizer = BERTTokenize()

train_dataset = PaperDataset(train_df, tokenizer)
val_dataset = PaperDataset(val_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=collate_fn)

## モデルの作成

In [ ]:
# 損失関数とオプティマイザーを定義して、モデルを作成します。
model = BERTModel()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BERTModel(
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=768, out_features=1, bias=True)
  (sig): Sigmoid()
)

## DeBART-X3の事前学習

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.5 MB/s eta 0:00:00


In [ ]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification
import torch
from torch.utils.data import DataLoader
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-xlarge')

In [ ]:
# データの前処理
# tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
import polars as pl
import torch
from transformers import DebertaTokenizer, DebertaForSequenceClassification

train_encodings = tokenizer(list(df_train["abstract"]), truncation=True, padding=True)
test_encodings = tokenizer(list(df_test["abstract"]), truncation=True, padding=True)
train_labels = torch.tensor(list(df_train["y"]))
test_labels = torch.tensor(list(df_test["y"]))
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

NameError: ignored

In [ ]:
df_test = df_test.with_columns(
    pl.DataFrame(np.array([0]*len(df_test)),columns="y")
)

In [ ]:
# 入力をトークン化し、エンコードする
def encode(examples):
    return tokenizer(examples[:,3], truncation=True, padding=True)

train_dataset = train_df.map(encode, batched=True)
test_dataset = test_df.map(encode, batched=True)